In [ ]:
import plotly.express as px
import pandas as pd

def skip_rows(index):
    return index % 20 != 0

# Load your dataset, skipping rows
df2023 = pd.read_csv('2023.prcp.us.csv', header=None, skiprows=lambda x: skip_rows(x))
df2022= pd.read_csv('2022.prcp.us.csv', header=None, skiprows=lambda x: skip_rows(x))
df=pd.concat([df2022,df2023.iloc[:,1:13]],axis=1)
df.columns = ['Latitude', 'Longitude'] + [f'Month_{i}' for i in range(1, 25)]

# Convert the dataset to long format for easier plotting with Plotly
df_long = df.melt(id_vars=['Latitude', 'Longitude'], var_name='Month', value_name='Precipitation')

# Create the interactive map
fig = px.scatter_geo(df_long,
                     lat='Latitude',
                     lon='Longitude',
                     color='Precipitation',
                     animation_frame='Month',
                     #scope='usa',  # Focus the map on the United States
                     projection="natural earth",
                     title="US 2023 Monthly Precipitation")

fig.update_geos(
    lataxis_range=[24.52, 49.38],  # Latitude range
    lonaxis_range=[-124.77, -66.95]  # Longitude range
)

fig.show()

In [3]:
import pandas as pd

# Load your datasets
# Replace 'your_actual_dataset.csv' and 'your_average_dataset.csv' with the actual file paths
df_actual_2022 = pd.read_csv('2022.prcp.us.csv', header=None, skiprows=lambda x: x % 20 != 0)
df_actual_2023 = pd.read_csv('2023.prcp.us.csv', header=None, skiprows=lambda x: x % 20 != 0)
df_average = pd.read_csv('us_prcp_1901-2000_normal.csv', header=None, skiprows=lambda x: x % 20 != 0)

# Assuming the first two columns are latitude and longitude, and the rest are the monthly values
# Ensure that both DataFrames have the same structure
df_actual_2022.columns = ['Latitude', 'Longitude'] + [f'Month_{i}' for i in range(1, 13)]
df_actual_2023.columns = ['Latitude', 'Longitude'] + [f'Month_{i}' for i in range(1, 13)]
df_average.columns = ['Latitude', 'Longitude'] + [f'Month_{i}' for i in range(1, 13)]

# Calculate the anomaly (actual - average)
df_anomaly_2022 = df_actual_2022.copy()
df_anomaly_2023 = df_actual_2023.copy()
df_anomaly_2022.iloc[:, 2:] = df_actual_2022.iloc[:, 2:] - df_average.iloc[:, 2:]*25.4
df_anomaly_2023.iloc[:, 2:] = df_actual_2023.iloc[:, 2:] - df_average.iloc[:, 2:]*25.4

# Convert the anomaly dataset to long format for easier plotting with Plotly
df_long_anomaly = df_anomaly_2022.melt(id_vars=['Latitude', 'Longitude'], var_name='Month', value_name='Anomaly')

# Rest of your processing and visualization code...


In [ ]:
custom_color_scale = [
    (0.0, "red"),            # Red for the lowest values
    #(0.19, "red"),           # Transition start from red to gray
    (0.3, "lightgray"),      # Light gray at 20th percentile
    (0.4, "gray"),           # Gray in the middle
    (0.6, "gray"),           # Gray in the middle
    (0.7, "lightgray"),      # Light gray at 80th percentile
    #(0.81, "blue"),          # Transition start from gray to blue
    (1.0, "blue")            # Blue for the highest values
]

# Create the interactive map with the custom color scale
figA = px.scatter_geo(df_long_anomaly,
                     lat='Latitude',
                     lon='Longitude',
                     color='Anomaly',
                     animation_frame='Month',
                     projection="natural earth",
                     color_continuous_scale=custom_color_scale,
                     title="US 2023 Monthly Precipitation Anomaly")

figA.update_geos(
    lataxis_range=[24.52, 49.38],  # Latitude range
    lonaxis_range=[-124.77, -66.95]  # Longitude range
)

figA.show()

In [5]:
import numpy as np

In [22]:
grid=0.2
mat=np.zeros(shape=(24,int((49.38-24.52)/grid)+1,int((-66.95+124.77)/grid)+1))#month,lat,lon
for ind,e in df_anomaly_2022.iterrows():
    for j in range(1,13):
        #print(e['Latitude'],e['Longitude'])
        mat[j-1][int((e['Latitude']-24.52)/grid)][int((e['Longitude']+124.77)/grid)]=max(mat[j-1][int((e['Latitude']-24.52)/grid)][int((e['Longitude']+124.77)/grid)],e[1+j])

for ind,e in df_anomaly_2023.iterrows():
    for j in range(1,13):
        #print(e['Latitude'],e['Longitude'])
        mat[j-1+12][int((e['Latitude']-24.52)/grid)][int((e['Longitude']+124.77)/grid)]=max(mat[j-1][int((e['Latitude']-24.52)/grid)][int((e['Longitude']+124.77)/grid)],e[1+j])

In [23]:
mat.shape

(24, 125, 290)

In [16]:
from pytrends.request import TrendReq
from datetime import datetime

pytrends = TrendReq(hl='en-US', tz=360)
kw_list = ["climate change"]

# Set the start date to the beginning of 2023 and the end date to today (or the last date in 2023 you're interested in)
start_date = "2022-01-01"
end_date = "2023-12-31"
#end_date = datetime.today().strftime('%Y-%m-%d')  # Today's date in YYYY-MM-DD format; adjust as needed

# Build the payload with the adjusted timeframe
pytrends.build_payload(kw_list, cat=0, timeframe=f'{start_date} {end_date}', geo='US', gprop='')

# Interest over time
interest_over_time_df = pytrends.interest_over_time()
print(interest_over_time_df)


            climate change  isPartial
date                                 
2022-01-02               2      False
2022-01-09               2      False
2022-01-16               1      False
2022-01-23               2      False
2022-01-30               2      False
...                    ...        ...
2023-12-03               3      False
2023-12-10               3      False
2023-12-17               1      False
2023-12-24               1      False
2023-12-31               1      False

[105 rows x 2 columns]


In [18]:
import pandas as pd
import numpy as np
from datetime import datetime

GTbyMonth = np.zeros(24)
for m in range(1, 13):
    # Generate the start and end date for each month
    start_date = datetime(2022, m, 1)
    end_date = pd.date_range(start_date, periods=1, freq='M').date[0]  # 'M' frequency stands for month end

    # Use the generated start and end dates to filter the DataFrame
    Monthlydf = interest_over_time_df.loc[start_date:end_date]

    # Sum the values for the month and assign to the corresponding position in GTbyMonth
    GTbyMonth[m-1] = int(Monthlydf['climate change'].sum())  # Adjusted index to m-1 since Python is zero-indexed

for m in range(1, 13):
    # Generate the start and end date for each month
    start_date = datetime(2023, m, 1)
    end_date = pd.date_range(start_date, periods=1, freq='M').date[0]  # 'M' frequency stands for month end

    # Use the generated start and end dates to filter the DataFrame
    Monthlydf = interest_over_time_df.loc[start_date:end_date]

    # Sum the values for the month and assign to the corresponding position in GTbyMonth
    GTbyMonth[m-1+12] = int(Monthlydf['climate change'].sum())  # Adjusted index to m-1 since Python is zero-indexed

print(GTbyMonth)


[  9.   8.   9. 114.  16.   6.   9.   8.  10.  15.  11.   9.  11.  11.
  11.  54.  12.   6.  10.   8.  11.  12.  11.   9.]


In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

# Corrected input shape to match the provided data
spatial_input = Input(shape=(mat.shape[1],mat.shape[2], 1))  # Shape adjusted to (height, width, channels)

# Example CNN architecture
conv1 = Conv2D(32, (3, 3), activation='relu')(spatial_input)
pool1 = MaxPooling2D((2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)
flat = Flatten()(pool2)
dense1 = Dense(128, activation='relu')(flat)
output = Dense(1, activation='linear')(dense1)  # Assuming a regression task

# Create the model with the corrected input shape
model = Model(inputs=spatial_input, outputs=output)

# Model summary to verify the input shape
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 125, 290, 1)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 123, 288, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 61, 144, 32)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 59, 142, 64)       18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 29, 71, 64)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 131776)            0   

In [25]:
X=mat
X_normalized = (X - X.min()) / (X.max() - X.min())
X_reshaped = X_normalized[..., np.newaxis]
model.compile(optimizer='adam', loss='mse') 

In [26]:
# Assuming X_spatial and X_temporal are your prepared spatial and temporal features, and y are your target labels
model.fit(x=X_reshaped,y=GTbyMonth, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 934.0715 - val_loss: 76.2454
Epoch 2/10
1/1 [==============================] - 0s 56ms/step - loss: 887.4233 - val_loss: 35.3732
Epoch 3/10
1/1 [==============================] - 0s 58ms/step - loss: 806.4567 - val_loss: 15.2439
Epoch 4/10
1/1 [==============================] - 0s 56ms/step - loss: 694.8845 - val_loss: 133.2050
Epoch 5/10
1/1 [==============================] - 0s 58ms/step - loss: 624.4861 - val_loss: 463.0757
Epoch 6/10
1/1 [==============================] - 0s 57ms/step - loss: 703.3589 - val_loss: 390.4833
Epoch 7/10
1/1 [==============================] - 0s 58ms/step - loss: 660.8423 - val_loss: 222.7180
Epoch 8/10
1/1 [==============================] - 0s 58ms/step - loss: 597.9500 - val_loss: 109.3720
Epoch 9/10
1/1 [==============================] - 0s 57ms/step - loss: 575.6867 - val_loss: 56.9601
Epoch 10/10
1/1 [==============================] - 0s 63ms/step - loss: 576.8725 - val_loss: 37.2